In [ ]:

# ? Se importan las librerías
import arcpy
import os
import pandas as pd
#? Geoaccesor para trabajar SEDF
from arcgis import GeoAccessor, GeoSeriesAccessor

# ? Pametrización de las rutas de salida, esto cambiará cuando se cree la clase
ruta_featureclass = r"D:\PUBLIC\PGAR\Resultados\4.PRO\PGAR\_12_Producto_Cartografico\Producto_Cartografico\Producto_Cartografico.gdb\_1_Recurso_Hidrico\Demanda_Hidrica_Cuenca"
# ? Esta variable debe ser también parametrizada como parámetro de la clase
ruta_archivos_salida = r"D:\PUBLIC\PGAR\Resultados\4.PRO\PGAR\_13_Calidad"
# ? Parámetros que se usarán a la hora de generar .xlsx resultante
nombre_archivo_xlsx = os.path.basename(ruta_featureclass)
salida_archivo_calidad = str(os.path.join(ruta_archivos_salida,nombre_archivo_xlsx)) + '.xlsx'

# ? Creación de SEDF
df_feature = pd.DataFrame.spatial.from_featureclass(location = ruta_featureclass)

# ? El atriubuto .shape genera una tupla con la información asociada a las dimensiones del SEDF
lista_estadisticos_generales = [[df_feature.shape[0], df_feature.shape[1]]]

# ? Se crea dataframe donde se almacenarán las dimensiones del SEDF
df_estadisticos_generales = pd.DataFrame(lista_estadisticos_generales, columns = ['numero_registros','numero_columnas'])

# ? Se almacena en una lista los nombres de las columnasy total de nulos.
# ? Para ello se recorre el SEDF por medio de la función .items()
# ! Existen otras formas que valdría la pena investigar
nombre_columna = [column_name for column_name, column_data in df_feature.items()]
total_nulos = [df_feature[column_name].isna().sum() for column_name, column_data in df_feature.items()]

# ? Como se está trabajando sobre un SEDF, al buscar blancos y analizar la columna geográfica, nos mostraba un error de GeoArray.
# ! Se debe investigar el tema de los GeoArray como objetos manipulables desde Python
# ? Para filtrar la columna se uso .loc, de la forma que se presenta a continuación y se almacena los resultados en nuevo dataframe
df_feature_sin_shape = df_feature.loc[:, df_feature.columns != 'SHAPE']
# ? Se almacenan los resultados en una lista
total_vacios = [((df_feature_sin_shape[column_name]=='').sum()) for column_name, column_data in df_feature_sin_shape.items()] 

# ? Se genera lista unificada con zip, para cada uno de los valores resultantes por columna
lista_resultados = list(zip(nombre_columna,total_nulos,total_vacios))
# ? Almaceno los resultados de la unificación de listas en un dataframe, el cual se expotará posteriormente como hoja a archivo excel
df_estadisticos_detallados = pd.DataFrame(lista_resultados, columns = ['nombre_columna','total_nulos','total_vacios'])

with pd.ExcelWriter(salida_archivo_calidad, engine='xlsxwriter') as writer:
    # ? Se utiliza el método to_excel para guardar cada DataFrame en una hoja separada.
    # ? Se puede especificar el nombre de la hoja con el argumento 'sheet_name'.
    df_estadisticos_generales.to_excel(writer, sheet_name='Est_Generales', index=False)
    df_estadisticos_detallados.to_excel(writer, sheet_name='Est_Detallados', index=False)  